<a href="https://colab.research.google.com/github/deenukhan/deep_learning/blob/main/2_2_writing_your_own_callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Notebook is inspired by Keras Official Documentation, Please checkout the below link for more information.
# https://keras.io/guides/writing_your_own_callbacks/#:~:text=Keras%20callbacks%20overview&text=Callback%20class%2C%20and%20override%20a,of%20the%20model%20during%20training.

## **Introduction**
A callback is a powerful tool to customize the behavior of a Keras model during training, evaluation, or inference. Examples include tf.keras.callbacks.TensorBoard to visualize training progress and results with TensorBoard, or tf.keras.callbacks.ModelCheckpoint to periodically save your model during training.

In this guide, you will learn what a Keras callback is, what it can do, and how you can build your own. We provide a few demos of simple callback applications to get you started.

## **Keras callbacks overview**
All callbacks subclass the keras.callbacks.Callback class, and override a set of methods called at various stages of training, testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training.

You can pass a list of callbacks (as the keyword argument callbacks) to the following model methods:

1. keras.Model.fit()
2. keras.Model.evaluate()
3. keras.Model.predict()

## **An overview of callback methods**
### **Global methods**
**on_(train|test|predict)_begin(self, logs=None)**
Called at the beginning of fit/evaluate/predict.

**on_(train|test|predict)_end(self, logs=None)**
Called at the end of fit/evaluate/predict.

### **Batch-level methods for training/testing/predicting**
**on_(train|test|predict)_batch_begin(self, batch, logs=None)**
Called right before processing a batch during training/testing/predicting.

**on_(train|test|predict)_batch_end(self, batch, logs=None)**
Called at the end of training/testing/predicting a batch. Within this method, logs is a dict containing the metrics results.

### **Epoch-level methods (training only)**
**on_epoch_begin(self, epoch, logs=None)**
Called at the beginning of an epoch during training.

**on_epoch_end(self, epoch, logs=None)**
Called at the end of an epoch during training.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
# We will be using below Model to Test out the callbacks 
keras.backend.clear_session()

fashion_mnist = keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

#Normalizing the data
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)
X_train, X_test = X_train/255. , X_test/255.

# Let's Create the Model

conv_model = keras.models.Sequential([
            keras.layers.Conv2D(128, kernel_size = 3, activation = 'relu', input_shape=(28, 28, 1)),
            keras.layers.MaxPooling2D((2,2)),
            keras.layers.Flatten(),
            keras.layers.Dense(512, activation = 'relu'),
            keras.layers.Dense(10, activation = 'softmax')
])

conv_model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
conv_model.fit(X_train, y_train, epochs=10, validation_split = 0.2)

Epoch 1/10
1500/1500 [==============================] - 40s 5ms/step - loss: 0.4908 - accuracy: 0.8253 - val_loss: 0.3002 - val_accuracy: 0.8910
Epoch 2/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2490 - accuracy: 0.9071 - val_loss: 0.2639 - val_accuracy: 0.9044
Epoch 3/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.1918 - accuracy: 0.9282 - val_loss: 0.2663 - val_accuracy: 0.9040
Epoch 4/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.1453 - accuracy: 0.9463 - val_loss: 0.2663 - val_accuracy: 0.9090
Epoch 5/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.1070 - accuracy: 0.9612 - val_loss: 0.2640 - val_accuracy: 0.9166
Epoch 6/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0806 - accuracy: 0.9708 - val_loss: 0.3040 - val_accuracy: 0.9147
Epoch 7/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0617 - accuracy: 0.9777 - val_loss: 0.3484 - val_accuracy

In [22]:
# We will be using below Model to Test out the callbacks 
# Let's Stop the Training when our Validation Accuracy has reached to some specific point

keras.backend.clear_session()

fashion_mnist = keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Below Code is used for Loss, and We can write multiple methods withing in single class
# class myCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs={}):
#     if(logs.get('val_accuracy')>0.86):
#       print("\nReached 60% accuracy so cancelling training!")
#       self.model.stop_training = True

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.86):
            print("\nValidation Accuracy is Good Now, Stopping the Trainging.......")
            self.model.stop_training = True

callback = myCallback()
#Normalizing the data
X_train, X_test = X_train/255. , X_test/255.

# Let's Create the Model
conv_model = keras.models.Sequential([
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation = 'relu'),
            keras.layers.Dense(10, activation = 'softmax')
])

conv_model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
conv_model.fit(X_train, y_train, epochs=10, validation_split = 0.2, callbacks=[callback])

Epoch 1/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6533 - accuracy: 0.7733 - val_loss: 0.4181 - val_accuracy: 0.8521
Epoch 2/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3929 - accuracy: 0.8597 - val_loss: 0.3815 - val_accuracy: 0.8648

Validation Accuracy is Good Now, Stopping the Trainging.......


## **Usage of self.model attribute**
In addition to receiving log information when one of their methods is called, callbacks have access to the model associated with the current round of training/evaluation/inference: self.model.

Here are of few of the things you can do with self.model in a callback:

1. Set self.model.stop_training = True to immediately interrupt training.
2. Mutate hyperparameters of the optimizer (available as self.model.optimizer), such as self.model.optimizer.learning_rate.
3. Save the model at period intervals.
4. Record the output of model.predict() on a few test samples at the end of each epoch, to use as a sanity check during training.
5. Extract visualizations of intermediate features at the end of each epoch, to monitor what the model is learning over time.
etc.

Let's see this in action in a couple of examples.

## **Learning rate scheduling**
In this example, we show how a custom Callback can be used to dynamically change the learning rate of the optimizer during the course of training.

See callbacks.LearningRateScheduler for a more general implementations.

In [24]:
# Define the Keras model to add callbacks to
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(1, input_dim=784))
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )
    return model

In [25]:
# Load example MNIST data and pre-process it
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0

# Limit the data to 1000 samples
x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

11493376/11490434 [==============================] - 0s 0us/step


In [27]:
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_epoch_end(self, epoch, logs=None):
        print(
            "The average loss for epoch {} is {:7.2f} "
            "and mean absolute error is {:7.2f}.".format(
                epoch, logs["loss"], logs["mean_absolute_error"]
            )
        )


class CustomLearningRateScheduler(keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))


LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05),
    (6, 0.01),
    (9, 0.005),
    (12, 0.001),
]


def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr


model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=15,
    verbose=0,
    callbacks=[
        LossAndErrorPrintingCallback(),
        CustomLearningRateScheduler(lr_schedule),
    ],
)


Epoch 00000: Learning rate is 0.1000.
For batch 0, loss is   23.47.
For batch 1, loss is  409.02.
For batch 2, loss is  281.10.
For batch 3, loss is  212.96.
For batch 4, loss is  171.80.
The average loss for epoch 0 is  171.80 and mean absolute error is    7.81.

Epoch 00001: Learning rate is 0.1000.
For batch 0, loss is    7.70.
For batch 1, loss is    6.22.
For batch 2, loss is    5.82.
For batch 3, loss is    5.74.
For batch 4, loss is    5.53.
The average loss for epoch 1 is    5.53 and mean absolute error is    1.89.

Epoch 00002: Learning rate is 0.1000.
For batch 0, loss is    4.79.
For batch 1, loss is    4.34.
For batch 2, loss is    4.43.
For batch 3, loss is    4.72.
For batch 4, loss is    4.66.
The average loss for epoch 2 is    4.66 and mean absolute error is    1.71.

Epoch 00003: Learning rate is 0.0500.
For batch 0, loss is    5.43.
For batch 1, loss is    4.19.
For batch 2, loss is    3.97.
For batch 3, loss is    3.89.
For batch 4, loss is    3.76.
The average loss